In [1]:
import os 
import numpy as np
import pandas as pd
import music21 as m
import json
import tensorflow.keras as keras

C:\Users\Adarsh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Adarsh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Adarsh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Adarsh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: P

In [2]:
from music21 import *
us = environment.UserSettings()
for key in sorted(us.keys()):
    key
us['musicxmlPath'] = 'C:\\Program Files\\MuseScore 3\\bin\\MuseScore3.exe'
us['musicxmlPath']


WindowsPath('C:/Program Files/MuseScore 3/bin/MuseScore3.exe')

In [22]:
midi_path="C:\\Users\\Adarsh\\Desktop\\New folder"
acceptable_durations={0.25,
                     0.5,
                     0.75,
                     1,
                     1.5,
                     2,
                     3,
                     4,6,8,18}
SAVE_DIR="C:\\Users\\Adarsh\\Desktop\\dataset"
dataset_path="C:\\Users\\Adarsh\\Desktop\\dataset"
single_file_dataset="File dataset"
sequence_length=256
mapping_path="mapping.json"

In [23]:
def load_midi_songs(path):
    songs=[]
    for path,subdirs,files in os.walk(path):
        for file in files:
            if file[-3:]=="mid":
                song=m.converter.parse(os.path.join(path,file))
                songs.append(song)
    return songs

In [24]:
def has_acceptable_durations(song, acceptable_durations):
    """Boolean routine that returns True if piece has all acceptable duration, False otherwise.
    :param song (m21 stream):
    :param acceptable_durations (list): List of acceptable duration in quarter length
    :return (bool):
    """
    for note in song.flat.notesAndRests:
        if note.duration.quarterLength not in acceptable_durations:
            return False
    return True


In [25]:
def encode_song(song, time_step=0.5):
   
    encoded_song = []

    for event in song.flat.notesAndRests:
        global symbol        
        if isinstance(event, m.note.Note):
            symbol = event.pitch.midi        
        elif isinstance(event, m.note.Rest):
            symbol = "r"       
        steps = int(event.duration.quarterLength / time_step)
        for step in range(steps):            
            if step == 0:
                encoded_song.append(symbol)
            else:
                encoded_song.append("_")    
    encoded_song = " ".join(map(str, encoded_song))

    return encoded_song

In [26]:
def preprocess(path):
    songs=load_midi_songs(midi_path)
    print(f"loaded {len(songs)}")
    for i,song in enumerate(songs):
        if not has_acceptable_durations(song, acceptable_durations):
            continue
        #song = transpose(song)
        encoded_song = encode_song(song)
        save_path = os.path.join(SAVE_DIR, str(i))
        with open(save_path, "w") as fp:
            fp.write(encoded_song)
                
           



In [27]:
def load(file_path):
    with open(file_path,"r") as fp:
        song=fp.read()
    return song

In [28]:
def create_dataset(dataset_path,file_dataset_path,sequence_length):
    new_song_deli="/ "*sequence_length
    songs=""
    for path,_,files in os.walk(dataset_path):
        for file in files:
            file_path=os.path.join(path,file)
            song=load(file_path)
            songs=songs+song+" "+new_song_deli
    songs=songs[:-1]
    with open(file_dataset_path,"w") as fp:
        fp.write(songs)
    return songs

In [29]:
def create_mapping(songs,mapping_path):
    mappings={}
    songs=songs.split()
    vocabulary=list(set(songs))
    for i,symbol in enumerate(vocabulary):
        mappings[symbol]=i
        with open(mapping_path,"w") as fp:
            json.dump(mappings,fp,indent=4)

In [30]:
def convert_songs_to_int(songs):
    int_songs=[]
    with open(mapping_path,"r") as fp:
        mappings=json.load(fp)
    songs=songs.split()
    for symbol in songs:
        int_songs.append(mappings[symbol])
    return int_songs

In [31]:
def generating_training_sequences(sequence_length):
    songs=load(single_file_dataset)
    int_songs=convert_songs_to_int(songs)
    inputs=[]
    targets=[]
    num_sequences=len(int_songs)-sequence_length
    for i in range(num_sequences):
        inputs.append(int_songs[i:i+sequence_length])
        targets.append(int_songs[i+sequence_length])
    vocabulary_size=len(set(int_songs))
    inputs=keras.utils.to_categorical(inputs,num_classes=vocabulary_size)
    targets=np.array(targets)
    return inputs,targets

In [32]:
if __name__=="__main__":
   
    preprocess(midi_path)
    songs=create_dataset(dataset_path,single_file_dataset,sequence_length)    
    create_mapping(songs,mapping_path)
    inputs ,targets = generating_training_sequences(sequence_length)
    
    #song.show()

loaded 14


In [33]:
inputs.shape


(1805, 256, 20)

In [34]:
targets.shape

(1805,)

In [35]:
#from preprocess import generating_training_sequences, sequence_length
import tensorflow.keras as keras
OUTPUT_UNITS=20
LOSS="sparse_categorical_crossentropy"
LEARNING_RATE=0.001
NUM_UNITS=[256,256]
EPOCHS=90
BATCH_SIZE=256
save_model_path="model.h5"

In [36]:
def build_model(output_units=OUTPUT_UNITS,num_units=NUM_UNITS,loss=LOSS,learning_rate=LEARNING_RATE):
    input=keras.layers.Input(shape=[None,output_units])
    x=keras.layers.LSTM(num_units[0])(input)
    x=keras.layers.Dropout(0.2)(x)
   
    
    output=keras.layers.Dense(output_units,activation="softmax")(x)
    model=keras.Model(input,output)
    model.compile(loss=loss,
                  optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  metrics=["accuracy"])
    model.summary()
    return model
    

In [37]:
def train(output_units,num_units,loss,learning_rate):
    inputs,targets=generating_training_sequences(sequence_length)
    model=build_model(output_units,num_units,loss,learning_rate)
    model.fit(inputs,targets,epochs=EPOCHS,batch_size=BATCH_SIZE)
    model.save(save_model_path)

In [38]:
if __name__== "__main__":
    train(output_units=OUTPUT_UNITS,num_units=NUM_UNITS,loss=LOSS,learning_rate=LEARNING_RATE)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, 20)]        0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               283648    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                5140      
Total params: 288,788
Trainable params: 288,788
Non-trainable params: 0
_________________________________________________________________
Epoch 1/90
1805/1805 [==============================] - 32s 18ms/sample - loss: 2.4002 - acc: 0.6737
Epoch 2/90
1805/1805 [==============================] - 35s 20ms/sample - loss: 0.9887 - acc: 0.7789
Epoch 3/90
1805/1805 [===========================

KeyboardInterrupt: 

In [39]:
class melodygenerator:
    def __init__(self,model_path="model.h5"):
        self.model_path=model_path
        self.model=keras.models.load_model(model_path)
        with open(mapping_path,"r") as fp :
            self.mappings=json.load(fp)
        self._start_symbols=["/"]*sequence_length
    
      
    def generate_melody(self,seed,num_steps,max_sequence_length,temperature):
        seed=seed.split()
        melody=seed
        seed=self._start_symbols+seed
        seed=[self.mappings[symbol] for symbol in seed]
        for _ in range(num_steps):
            seed = seed[-max_sequence_length:]
            onehot_seed=keras.utils.to_categorical(seed,num_classes=len(self.mappings))
            onehot_seed=onehot_seed[np.newaxis,...]
            probabilities = self.model.predict(onehot_seed)[0]
            output_int = self._sample_with_temperature(probabilities, temperature)
            seed.append(output_int)
            output_symbol = [k for k, v in self.mappings.items() if v == output_int][0]
            if output_symbol == "/":
                break

            # update melody
            melody.append(output_symbol)

        return melody
    def _sample_with_temperature(self, probabilites, temperature):
        
        predictions = np.log(probabilites) / temperature
        probabilites = np.exp(predictions) / np.sum(np.exp(predictions))

        choices = range(len(probabilites)) # [0, 1, 2, 3]
        index = np.random.choice(choices, p=probabilites)

        return index
    def save_melody(self, melody, step_duration=0.25, format="midi", file_name="mel.mid"):
     
        # create a music21 stream
        stream = m.stream.Stream()

        start_symbol = None
        step_counter = 1

        # parse all the symbols in the melody and create note/rest objects
        for i, symbol in enumerate(melody):

            # handle case in which we have a note/rest
            if symbol != "_" or i + 1 == len(melody):

                # ensure we're dealing with note/rest beyond the first one
                if start_symbol is not None:

                    quarter_length_duration = step_duration * step_counter # 0.25 * 4 = 1

                    # handle rest
                    if start_symbol == "r":
                        m21_event = m.note.Rest(quarterLength=quarter_length_duration)

                    # handle note
                    else:
                        m21_event = m.note.Note(int(start_symbol), quarterLength=quarter_length_duration)

                    stream.append(m21_event)

                    # reset the step counter
                    step_counter = 1

                start_symbol = symbol

            # handle case in which we have a prolongation sign "_"
            else:
                step_counter += 1

        # write the m21 stream to a midi file
        stream.write(format, file_name)

    

In [40]:
if __name__ == "__main__":
    mg = melodygenerator()
    seed = "67 _ 67 _ 67 _ _ 65 64 _ 64 _ 64 _ _"
    seed2 = "67 _ _ _ _ _ 65 _ 64 _ 62 _ 60 _ _ _"
    melody = mg.generate_melody(seed2, 2000, sequence_length, 0.1)
    print(melody)
    mg.save_melody(melody)

['67', '_', '_', '_', '_', '_', '65', '_', '64', '_', '62', '_', '60', '_', '_', '_', '72', '_', '72', '_', '72', '_', '72', '_', '72', '_', '69', '_', '_', '_', '_', '_', 'r', '_', 'r', '_', '72', '_', '72', '_', '72', '_', '74', '_', '_', '_']
